# Baixa os dados a cada 20s do GLM do satelite GOES
 - Dados GLM GOES-19 fornecido pela AMAZON: https://noaa-goes19.s3.amazonaws.com/index.html#GLM-L2-LCFA/
 - Realizado por: Enrique V. Mattos - 25/09/2025

# **1° Passo:** Preparando ambiente

In [ ]:
# instala e importa bibliotecas
!pip install -q boto3

# importa bibliotecas
import os
import time
from datetime import timedelta, datetime

# monta drive
from google.colab import drive
drive.mount('/content/drive')

# diretórios
dir = '/content/drive/MyDrive/2-PESQUISA/0_GLM/estudos_de_caso/2025-09-22-FRENTEFRIA_RS_SUDESTE'
dir_output = f'{dir}/output/glm_20s'
os.makedirs(dir_output, exist_ok=True)

# **2° Passo:** Declarando funções

In [ ]:
def download_GLM(yyyymmddhhmnss, path_dest):

    # importando bibliotecas
    import boto3                             # Amazon Web Services (AWS) SDK for Python
    from botocore import UNSIGNED            # boto3 config
    from botocore.config import Config       # boto3 config

    # extrai a data
    year = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%Y')
    day_of_year = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%j')
    hour = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%H')
    min = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%M')
    seg = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%S')

    # FTP da AMAZON
    # Caminho: https://noaa-goes19.s3.amazonaws.com/index.html
    # Caminho básico: https://noaa-goes16.s3.amazonaws.com/index.html#GLM-L2-LCFA/
    # Caminho completo: https://noaa-goes19.s3.amazonaws.com/index.html#GLM-L2-LCFA/2025/016/15/
    bucket_name = 'noaa-goes19'

    # inicializa o S3 client
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

    # estrutura do arquivo
    product_name = "GLM-L2-LCFA"
    prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}_G19_s{year}{day_of_year}{hour}{min}{seg}'

    # pesquisa para o arquivo noi servidor
    s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

    # verifica se o arquivo não esta disponivel
    if 'Contents' not in s3_result:

        # There are no files
        print(f'No files found for the date: {yyyymmddhhmnss}, Product-{product_name}')
        return -1

    # se ele estiver disponivel
    else:

        for obj in s3_result['Contents']:

            key = obj['Key']
            file_name = key.split('/')[-1].split('.')[0]

            # Download the file
            if os.path.exists(f'{path_dest}/{file_name}.nc'):
                print(f'File {path_dest}/{file_name}.nc exists')

            else:
                print(f'Downloading file {path_dest}/{file_name}.nc')
                s3_client.download_file(bucket_name, key, f'{path_dest}/{file_name}.nc')

    return f'{file_name}'

# **Processando os dados**

 - Em **1 hora** de dados temos `180 arquivos` e demora `6 min` para baixar.

 - Para 1 dia vai demorar `144min`.

In [ ]:
%%time
#========================================================================================================================#
#                                                 DEFINIÇÕES INICIAIS
#========================================================================================================================#
# data
yyyymmddhhmn = '202509210000'

# área desejada
lonmin, lonmax, latmin, latmax = -65, -35, -35, -10

#========================================================================================================================#
#                                                 DOWNLOAD DOS DADOS
#========================================================================================================================#
# ano, mes, dia, hora, minuto e segundos
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

# cria as pastas de saida: ano-mes-dia/hora
for h in range(24):
    dir_output_data = f'{dir_output}/{yyyy}-{mm}-{dd}/{str(h).zfill(2)}'
    os.makedirs(dir_output_data, exist_ok=True)

In [ ]:
%%time
# data inicial
date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))

# data final (inicial + 24h)
date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(hours=24))
date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(hours=1))

print('#------------------------------------------#')
print('PROCESSANDO DATAS')
print(date_ini)
print(date_end)
print('#------------------------------------------#', '\n')

# data inicia é fixada como a data do loop
date_loop = date_ini

# loop nos arquivos do GLM
while (date_loop <= date_end):

    #-----------------------------------------------------------------#
    #                    BAIXANDO OS DADOS
    #-----------------------------------------------------------------#
    # data
    yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # ano, mes, dia, hora, minuto e segundos
    ano = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y')
    mes = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%m')
    dia = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%d')
    hor = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%H')
    min = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%M')
    seg = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%S')
    print(f'{ano}-{mes}-{dia} {hor}:{min}:{seg}')

    # caminho + data
    dir_output_data = f'{dir_output}/{ano}-{mes}-{dia}/{hor}'

    # download o arquivo
    file_glm = download_GLM(yyyymmddhhmnss, dir_output_data)

    # incrementa a variável the date_loop
    date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))